## Combination Technique

In [3]:
# Compare the numerical and analytical solutions 

# Define functions that evaluate the Dirichlet and Neumann trace of the
# analytical solution, expressed as a series of spherical wave functions
# Done for an exterior sound-soft and soft-hard scattering problem by a radius 1 sphere. 
# Could be adapted to transmission problem
# Only working for R = 1

from __future__ import division
import numpy as np
from scipy.special import sph_jn, sph_yn, lpn
from projection import discrete_coefficients_projection, refine, get_h
import scipy.linalg


kExt = 0.4
kInt = 1
rhoExt = 1
rhoInt = 1
R = 1

alpha = rhoInt / rhoExt

l_max = 200
l = np.arange(0, l_max + 1)

jExt,djExt = sph_jn(l_max, kExt * R)

yExt, dyExt = sph_yn(l_max, kExt * R)


hExt, dhExt = jExt + 1j * yExt, djExt + 1j * dyExt


jInt,djInt = sph_jn(l_max, kInt * R)

aInc = (2 * l + 1) * 1j ** l


cInt = ((kExt / rhoExt * (hExt * djExt - dhExt * jExt)) /
        ((kInt / rhoInt * hExt * djInt - kExt / rhoExt * dhExt * jInt))) * aInc

for l in range(l_max + 1):
    if abs(cInt[l] * jInt[l]) < 1e-16:
        # neglect all further terms
        l_max = l - 1
        aInc = aInc[:l]
        cInt = cInt[:l]
        break

def uExactDirichletTrace(point):
    x, y, z = point
    r = np.sqrt(x**2 + y**2 + z**2)
    jInt = sph_jn(l_max, kInt * r)[0]
    Y, dY = lpn(l_max, x / r)
    return (cInt * jInt * Y).sum()

def uExactIntNeumannTrace(point):
    x, y, z = point
    r = np.sqrt(x**2 + y**2 + z**2)
    n_x, n_y, n_z = point / r
    
    jInt, djInt = sph_jn(l_max, kInt * r)

    Y, dY = lpn(l_max, x / r)
    return (
         kInt * (cInt * djInt * Y).sum() * (n_x * x + n_y * y + n_z * z)+
        (cInt * jInt * dY).sum() *
        (-n_x * x * z - n_y * y * z + n_z * (x * x + y * y)) / (r * r * r)) 

def uExactExtNeumannTrace(point):
    return rhoExt / rhoInt * uExactIntNeumannTrace(point)

def uNE(x, n, domain_index, result):
    result[0] = uExactIntNeumannTrace(x)

def uDE(x, n, domain_index, result):
    result[0] = uExactDirichletTrace(x)

def rescale(A, alpha):
    """Rescale the 2x2 block operator matrix A"""
    A[0, 1] = A[0, 1] * alpha
    A[1, 0] = A[1, 0] * (1/alpha)
    return A


/home/bempp/.local/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in true_divide


In [5]:

import bempp.api as bem
import scipy 
L20 = []
L21 = []

H0 = []
H1 = []

h_list =[]

bem.global_parameters.hmat.max_block_size = 1000
bem.global_parameters.hmat.eps = 1e-8#0.00001



def dirichlet_fun(x, n, domain_index, result):
    result[0] = np.exp(1j * kExt * x[0])

def neumann_fun(x, n, domain_index, result):
    result[0] = 1j * kExt * np.exp(1j * kExt * x[0]) * n[0]



#bem.global_parameters.assembly.boundary_operator_assembly_type = 'dense'
print bem.global_parameters.assembly.boundary_operator_assembly_type

nQuad = 3
bem.global_parameters.quadrature.near.single_order = nQuad+2
bem.global_parameters.quadrature.near.double_order = nQuad+2
bem.global_parameters.quadrature.medium.single_order = nQuad+1
bem.global_parameters.quadrature.medium.double_order = nQuad+1
bem.global_parameters.quadrature.far.single_order = nQuad
bem.global_parameters.quadrature.far.double_order = nQuad

name = '../Meshes/L0.msh'
grid = bem.import_grid(name)

L = 2
#precision_list = [10,15, 20, 25, 30, 35, 40]
for l in range(L+1):
    space = bem.function_space(grid, "P", 1)
    print int(space.global_dof_count), 'NP1'
    h = get_h(grid)[2]
    
    dirichlet_grid_fun = bem.GridFunction(space, fun=dirichlet_fun)
    neumann_grid_fun = bem.GridFunction(space, fun=neumann_fun)


    x = [None, None]
    #x[0] = np.loadtxt('TP_results/x0_%i_kInt_%f_kExt_%f.txt' %(l, kInt, kExt)).view(complex)
    #x[1] = np.loadtxt('TP_results/x1_%i_kInt_%f_kExt_%f.txt' %(l, kInt, kExt)).view(complex)

    x[0] = np.loadtxt('TP_results/x0_%i.txt' %l).view(complex)
    x[1] = np.loadtxt('TP_results/x1_%i.txt' %l).view(complex)

    uExt = bem.GridFunction(space, coefficients = x[0])
    uExtDeriv = bem.GridFunction(space, coefficients = x[1])


    uSc = uExt-dirichlet_grid_fun
    uScDeriv = uExtDeriv- neumann_grid_fun

    uInt = uExt
    uIntDeriv = rhoInt / rhoExt * uExtDeriv

    # Error 
    uIntDerivRef = bem.GridFunction(space, fun = uNE)
    uIntRef = bem.GridFunction(space, fun = uDE)
    l20 = uInt.relative_error(uExactDirichletTrace)
    l21 = uIntDeriv.relative_error(uExactIntNeumannTrace)
    
    L20.append(l20)
    L21.append(l21)

    P2 = bem.function_space(grid, 'P', 2)
    W2 = bem.operators.boundary.laplace.hypersingular(P2, P2, P2)
    W2_wf = W2.weak_form()
    
    V2 = bem.operators.boundary.laplace.single_layer(P2, P2, P2)
    V2_wf = V2.weak_form()

    from bempp.api.space.projection import discrete_coefficient_projection
    P12 = discrete_coefficient_projection(space, P2)

    uP20 = bem.GridFunction(P2, coefficients=P12.matvec(uInt.coefficients))
    u0 = bem.GridFunction(P2, fun=uDE)

    uP21 = bem.GridFunction(P2, coefficients=P12.matvec(uIntDeriv.coefficients))
    u1 = bem.GridFunction(P2, fun=uNE)



    Sr0 = np.dot(np.array([u0.coefficients]).T, np.array([u0.coefficients]))
    Sr1 = np.dot(np.array([u1.coefficients]).T, np.array([u1.coefficients]))


    S0 = np.dot(np.array([uP20.coefficients]).T, np.array([uP20.coefficients]))
    S1 = np.dot(np.array([uP21.coefficients]).T, np.array([uP21.coefficients]))

    Err0 = S0 - Sr0
    Err1 = S1 - Sr1

    mv1 = W2_wf.matvec(uP20.coefficients)
    mv2 = W2_wf.matvec(u0.coefficients)
    S0b = np.dot(np.array([mv1]).T, np.array([mv1]))-np.dot(np.array([mv2]).T, np.array([mv2]))
    h0 = np.sqrt(np.abs(np.vdot(np.ravel(Err0), np.ravel(S0b))))

    mv1 = V2_wf.matvec(uP21.coefficients)
    mv2 = V2_wf.matvec(u1.coefficients)
    S1b = np.dot(np.array([mv1]).T, np.array([mv1]))-np.dot(np.array([mv2]).T, np.array([mv2]))
    h1 = np.sqrt(np.abs(np.vdot(np.ravel(Err1), np.ravel(S1b))))

    mv1 = W2_wf.matvec(u0.coefficients)
    S0b = np.dot(np.array([mv1]).T, np.array([mv1]))
    Rel0 = np.sqrt(np.abs(np.vdot(np.ravel(Sr0), np.ravel(S0b))))

    mv1 = V2_wf.matvec(u1.coefficients)
    S1b = np.dot(np.array([mv1]).T, np.array([mv1]))
    Rel1 = np.sqrt(np.abs(np.vdot(np.ravel(Sr1), np.ravel(S1b))))

    H0.append(h0 / Rel0)
    H1.append(h1 / Rel1)
    print '-------------'
    print h, ' =:h'
    print h0, Rel0, '0'
    print h1, Rel1, '0'
    print '-------------'
    h_list.append(h)
    #grid.plot()
    name = '../Meshes/L%i.msh' %(l+1)
    if l < L+1:
        grid = bem.import_grid(name)

output = [H0, H1]

file_output = open("TP_2_error.txt","a")
file_output.write(str(output)+"\r\n")
file_output.flush()
file_output.close() 


hmat
26 NP1
-------------
0.763541028701  =:h
0.0423618568575 0.403661147501 0
0.186704675582 1.88209695136 0
-------------
50 NP1
-------------
0.564007382607  =:h
0.913476706539 0.447336982617 0
0.834005995232 2.35685799746 0
-------------
98 NP1
-------------
0.407527008286  =:h
1.16050719475 0.472290464416 0
0.834312848077 2.63638271716 0
-------------
